In [1]:
# Import required libraries and dependencies
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import hvplot.pandas
import os
os.environ["OMP_NUM_THREADS"] = "1"
from joblib import parallel_backend
import warnings


In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
crypto_scaler = StandardScaler().fit_transform(df_market_data)
crypto_scaler[0:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [6]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(crypto_scaler, columns=['price_change_percentage_24h', 'price_change_percentage_7d', 'price_change_percentage_14d', 'price_change_percentage_30d', 'price_change_percentage_60d', 'price_change_percentage_200d',	'price_change_percentage_1y'])
df_market_data_scaled

# Copy the crypto names from the original DataFrame
coins = list(df_market_data.index) #list of all the coin names from the original data
coins

# Set the coin_id column as index
df_market_data_scaled['coin_id'] = coins  #new column containing the list of coins
df_market_data_scaled = df_market_data_scaled.set_index('coin_id') #set the df index to the new coin_id column

# Display the scaled DataFrame
df_market_data_scaled.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [7]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [8]:
# Suppress specific UserWarnings related to KMeans memory leak on Windows
warnings.filterwarnings("ignore", category=UserWarning, message=".*KMeans is known to have a memory leak.*")

# Set OMP_NUM_THREADS to 1 to avoid the MKL memory leak issue on Windows
os.environ["OMP_NUM_THREADS"] = "1"

# Set the max number of CPU cores for loky (optional, if needed)
os.environ["LOKY_MAX_CPU_COUNT"] = "4"

# Create a list for k-values (k from 1 to 11)
k = list(range(1, 12))

# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
    # Create a KMeans model using the loop counter for the n_clusters
    k_model = KMeans(n_clusters=i, random_state=1, n_init=10)  # Removed n_jobs=-1
    
    # Fit the model to the scaled data (df_market_data_scaled)
    k_model.fit(df_market_data_scaled)  # Ensure df_market_data_scaled is preprocessed data
    
    # Append the model's inertia value to the inertia list
    inertia.append(k_model.inertia_)

# Now, inertia holds the inertia values for each k
print(inertia)

[286.99999999999994, 195.82021818036043, 123.19048183836959, 79.02243535120975, 63.85866780584266, 53.05778846567062, 44.40679058461641, 37.07823336746088, 32.83218742836354, 28.165433305979253, 24.9646378775424]


In [9]:

# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

df_elbow

,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,63.858668
5,6,53.057788
6,7,44.406791
7,8,37.078233
8,9,32.832187
9,10,28.165433


In [10]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve Crypto", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The value for K is 4. 

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [11]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [12]:

# Fit the K-Means model using the scaled DataFrame
model.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=1)

In [13]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
clusters = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(clusters)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [14]:

# Create a copy of the scaled DataFrame
df_market_data_scaled_copy = df_market_data_scaled.copy()


In [15]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_market_data_scaled_copy["Predicted Cluster"] = clusters

# Display the copy of the scaled DataFrame
df_market_data_scaled_copy.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Predicted Cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [16]:

# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_market_data_scaled_copy.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='Predicted Cluster',
    hover_cols = 'coin_id')


:NdOverlay   [Predicted Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [17]:
# Create a PCA model instance and set `n_components=3`.
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [18]:

# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
df_pca = pca.fit_transform(df_market_data_scaled)

# View the scaled PCA data
df_pca[0:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [19]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The explained variances of the components are 0.3719856 + 0.34700813 + 0.17603793 = 0.89688183(89.7%)


In [20]:
# Create a new DataFrame with the PCA data.
df_market_pca = pd.DataFrame(
    df_pca,
    columns =["PC1", "PC2", "PC3"]
)
# Copy the crypto names from the original scaled DataFrame
df_market_pca["coin_id"] = df_market_pca.index

# Set the coin_id column as index
df_market_pca = df_market_pca.set_index("coin_id")

# Display the scaled PCA DataFrame
df_market_pca.head()

,PC1,PC2,PC3
coin_id,,,
0,-0.600667,0.842760,0.461595
1,-0.458261,0.458466,0.952877
2,-0.433070,-0.168126,-0.641752
3,-0.471835,-0.222660,-0.479053
4,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [21]:
# Create a list with the number of k-values from 1 to 11
k2= list(range(1,12))
k2

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [22]:
# Create an empty list to store the inertia values
inertia2 = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
for i in k2:
    k_model = KMeans(n_clusters=i, random_state=1, n_init=10)#Using n_init=10 to avoind FutureWarning: The default value of `n_init` warning
# 2. Fit the model to the data using `df_market_data_pca`
    k_model.fit(df_market_pca)
# 3. Append the model.inertia_ to the inertia list
    inertia2.append(k_model.inertia_)

In [23]:

# Create a dictionary with the data to plot the Elbow curve
elbow_data2 = {"k": k, "inertia": inertia2}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow2 = pd.DataFrame(elbow_data2)

df_elbow2


,k,inertia
0,1,256.874086
1,2,165.901994
2,3,93.774626
3,4,49.665497
4,5,38.072792
5,6,27.720626
6,7,21.145312
7,8,17.208546
8,9,13.719376
9,10,10.559358


In [24]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_curve_2= df_elbow2.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve PCA", 
    xticks=k2
)
elbow_curve_2

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**The best value for K is 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It is still 4. If the best 𝑘 for the original scaled data was a different value (e.g., 3 or 5), then the best 𝑘 for the PCA data does differ from the original data

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [25]:
# Initialize the K-Means model using the best value for k
model2 = KMeans(n_clusters=4, random_state=1)

In [26]:
model2.fit(df_market_pca)

KMeans(n_clusters=4, random_state=1)

In [27]:

# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
clusters2 = model2.predict(df_market_pca)

# Print the resulting array of cluster values.
print(clusters2)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [28]:
# Create a copy of the scaled PCA DataFrame
df_market_pca_copy= df_market_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_market_pca_copy['Clusters Predition'] = clusters2

# Display the copy of the scaled PCA DataFrame
df_market_pca_copy.head()

,PC1,PC2,PC3,Clusters Predition
coin_id,,,,
0,-0.600667,0.842760,0.461595,2
1,-0.458261,0.458466,0.952877,2
2,-0.433070,-0.168126,-0.641752,0
3,-0.471835,-0.222660,-0.479053,0
4,-1.157800,2.041209,1.859715,2


In [29]:

# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_market_pca_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    by = 'Clusters Predition',
    hover_cols='coin_id'
)

:NdOverlay   [Clusters Predition]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [30]:
# Composite plot to contrast the Elbow curves
df_elbow.hvplot.line(x="k", y="inertia", title= "Elbow Curve Crypto", xticks=k) + df_elbow2.hvplot.line(x="k", y="inertia", title= "Elbow Curve PCA",xticks=k)

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [31]:
# Composite plot to contrast the clusters
df_market_data_scaled_copy.hvplot.scatter( x='price_change_percentage_24h', y='price_change_percentage_7d',  title= "Cluster Crypto", by='Predicted Cluster', hover_cols = 'coin_id') + df_market_pca_copy.hvplot.scatter(x="PC1", y="PC2",title= "Cluster CPA", by = 'Clusters Predition', hover_cols='coin_id')

:Layout
   .NdOverlay.I  :NdOverlay   [Predicted Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Clusters Predition]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Based on the analysis, it seems that using PCA data to cluster the cryptocurrency data using K-Means has had a positive impact. The results show in a more defined and separable clusters compared to using the original data. Utilizing the PCA has reduced the numbers of features which has helped to highlight the essential patterns and reduce the impact of noise, leading to more accurate and meaningful clustering results.